# Multiple-Choice Question Answering

In [1]:
!nvidia-smi
!lscpu

Sat Aug 14 14:14:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0
!pip install editdistance==0.5.3

     |████████████████████████████████| 316 kB 13.9 MB/s 
     |████████████████████████████████| 1.4 MB 30.1 MB/s 
     |████████████████████████████████| 233 kB 45.5 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=46bec6812e2ee6e93e5c4b235eb9c308e3c3a19b13cbad9ceb8033c588c2ec3d
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154588 sha256=1318309ae287bb8ff28cdcbd02a2ef2609b4d59cca25ecec7410113036dd6896
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 43 kB 1.7 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import json
import collections
import numpy as np
import pandas as pd
import editdistance

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForMultipleChoice
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer
from transformers.data.metrics.squad_metrics import compute_exact, compute_f1

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [5]:
class MultipleChoiceQADataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Multiple Choice Question Answering. """

    def __init__(self, questions, candidates, choices, answers, tokenizer, max_length, model_type):
        self.questions = questions
        self.candidates = candidates
        self.choices = choices
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.model_type = model_type

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, item):
        if self.model_type == "mt5":
            input_text = self.questions[item] + ' <sep> ' + ' <sep> '.join(self.candidates[item])
            encoding = self.tokenizer(
                input_text,
                add_special_tokens=True,
                max_length=self.max_length,
                truncation=True,
                padding='max_length',
                return_tensors="pt"
            )
            inputs = {
                'item': str(item),
                'question': self.questions[item],
                'candidates': ' <sep> '.join(self.candidates[item]),
                'input_text': input_text,
                'choice': self.choices[item],
                'answer': self.answers[item],
                'input_ids': encoding.input_ids.flatten(),
                'attention_mask': encoding.attention_mask.flatten()
            }
            return inputs
        else:
            choices_input_ids, choices_attention_masks, choices_token_type_ids = [], [], []
            for c in self.candidates[item]:
                text_a = ""  # empty context
                text_b = self.questions[item] + " " + c
                inputs = self.tokenizer(
                    text_a,
                    text_b,
                    add_special_tokens=True,
                    max_length=self.max_length,
                    padding="max_length",
                    truncation=True,
                    return_overflowing_tokens=True
                )
                choices_input_ids.append(inputs.input_ids[0])
                choices_attention_masks.append(inputs.attention_mask[0])
                choices_token_type_ids.append(inputs.token_type_ids[0])

            inputs = {
                'item': str(item),
                'question': self.questions[item],
                'candidates': ' <sep> '.join(self.candidates[item]),
                'choice': int(self.choices[item]) - 1,
                'answer': self.answers[item],
                'input_ids': torch.LongTensor(choices_input_ids),
                'attention_mask': torch.LongTensor(choices_attention_masks),
                'token_type_ids': torch.LongTensor(choices_token_type_ids)
            }
            return inputs


class MultipleChoiceQA:
    def __init__(self, model_name, model_type):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        if model_type.lower() == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        elif model_type.lower() in ["mbert", "parsbert", "wikibert"]:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.model = AutoModelForMultipleChoice.from_pretrained(self.model_name, config=self.config)
            self.model_type = model_type.lower()
        else:
            print(f'model_type not supported!')
            return

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() in ["parsinlu", "parsinlu-literature", "parsinlu-math_and_logic",
                                    "parsinlu-common_knowledge"]:
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            questions, candidates, choices, answers = [], [], [], []
            with open(dataset_file, encoding="utf8") as infile:
                for line in infile:
                    json_line = json.loads(line.strip())
                    question = json_line['question']
                    candidate_answers = json_line['candidates']
                    choice = json_line['answer']
                    answer = candidate_answers[int(json_line['answer']) - 1]

                    questions.append(question)
                    candidates.append(candidate_answers)
                    choices.append(choice)
                    answers.append(answer)
            return questions, candidates, choices, answers

    def multiple_choice_qa_inference(self, questions, candidates, device, max_length=512):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        input_ids, attention_masks, token_type_ids = [], [], []
        for q, cs in zip(questions, candidates):
            choices_input_ids, choices_attention_masks, choices_token_type_ids = [], [], []
            for c in cs:
                text_a = ""  # empty context
                text_b = q + " " + c
                inputs = self.tokenizer(
                    text_a,
                    text_b,
                    add_special_tokens=True,
                    max_length=max_length,
                    padding="max_length",
                    truncation=True,
                    return_overflowing_tokens=True,
                )
                choices_input_ids.append(inputs.input_ids[0])
                choices_attention_masks.append(inputs.attention_mask[0])
                choices_token_type_ids.append(inputs.token_type_ids[0])
            input_ids.append(choices_input_ids)
            attention_masks.append(choices_attention_masks)
            token_type_ids.append(choices_token_type_ids)

        input_ids = torch.LongTensor(input_ids).to(device)
        attention_masks = torch.LongTensor(attention_masks).to(device)
        token_type_ids = torch.LongTensor(token_type_ids).to(device)

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        outputs = self.model(input_ids=input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
        predictions = torch.argmax(outputs.logits, dim=1)
        return [(questions[i], candidates[i], candidates[i][p.item()]) for i, p in enumerate(predictions)]

    def mt5_multiple_choice_qa_inference(self, questions, candidates, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        new_input = []
        for q, cs in zip(questions, candidates):
            new_input.append(q + ' <sep> ' + ' <sep> '.join(cs))

        tokenized_batch = self.tokenizer(
            new_input,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)

        outputs = self.model.generate(input_ids=input_ids, attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return [(questions[i], candidates[i], p) for i, p in enumerate(predictions)]

    def evaluation(self, questions, candidates, choices, answers, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(questions) != len(candidates):
            print('length of two inputs is not equal!!')
            return
        if len(choices) != len(answers):
            print('length of choices and answers is not equal!!')
            return
        if len(questions) != len(answers):
            print('length of inputs and answers is not equal!!')
            return

        dataset = MultipleChoiceQADataset(questions=questions, candidates=candidates, choices=choices, answers=answers,
                                          tokenizer=self.tokenizer, max_length=max_length, model_type=self.model_type)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#question:{len(questions)}, #candidates:{len(candidates)}, #answer:{len(answers)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_loss, total_time = 0, 0
        output_predictions = []
        golden_choices, predicted_choices = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']
            b_token_type_ids = batch['token_type_ids']
            b_choices = batch['choice']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_choices = b_choices.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                                       token_type_ids=b_token_type_ids, labels=b_choices)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')
            # get the loss
            total_loss += b_outputs.loss.item()

            golden_choices.extend(b_choices.cpu().detach().numpy().tolist())
            b_predictions = torch.argmax(b_outputs.logits, dim=1)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            predicted_choices.extend(b_predictions)

            for i in range(len(b_input_ids)):
                output_predictions.append((
                    batch['question'][i],
                    batch['candidates'][i].split(' <sep> '),
                    batch['choice'][i].item(),
                    batch['answer'][i],
                    b_predictions[i],
                    batch['candidates'][i].split(' <sep> ')[b_predictions[i]]
                ))

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(data_loader)
        print("average loss:", avg_train_loss)

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(questions))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_choices, predicted_choices)))
        print("Test Precision: {}".format(precision_score(golden_choices, predicted_choices, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_choices, predicted_choices, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_choices, predicted_choices, average="weighted")))
        print("Test classification Report:\n{}".format(
            classification_report(golden_choices, predicted_choices, digits=10)))
        return output_predictions

    def mt5_evaluation(self, questions, candidates, choices, answers, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(questions) != len(candidates):
            print('length of two inputs is not equal!!')
            return
        if len(choices) != len(answers):
            print('length of choices and answers is not equal!!')
            return
        if len(questions) != len(answers):
            print('length of inputs and answers is not equal!!')
            return

        dataset = MultipleChoiceQADataset(questions=questions, candidates=candidates, choices=choices, answers=answers,
                                          tokenizer=self.tokenizer, max_length=max_length, model_type="mt5")
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#question:{len(questions)}, #candidates:{len(candidates)}, #answer:{len(answers)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_time = 0
        output_predictions = []
        golden_choices, predicted_choices, exact_score_list, f1_score_list = [], [], [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)

            for i in range(len(b_input_ids)):
                if b_predictions[i] in batch['candidates'][i].split(' <sep> '):
                    predicted_choice = str(batch['candidates'][i].split(' <sep> ').index(b_predictions[i]) + 1)
                else:
                    normalized_edit_distance_list = [
                        editdistance.distance(ca, b_predictions[i]) / max(len(ca), len(b_predictions[i])) for ca in
                        batch['candidates'][i].split(' <sep> ')
                    ]
                    predicted_choice = str(normalized_edit_distance_list.index(min(normalized_edit_distance_list)) + 1)

                golden_choices.append(batch['choice'][i])
                predicted_choices.append(predicted_choice)

                exact_score_list.append(compute_exact(batch['answer'][i], b_predictions[i]))
                f1_score_list.append(compute_f1(batch['answer'][i], b_predictions[i]))

                output_predictions.append((
                    batch['question'][i],
                    batch['candidates'][i].split(' <sep> '),
                    batch['choice'][i],
                    batch['answer'][i],
                    predicted_choice,
                    b_predictions[i],
                    exact_score_list[-1],
                    f1_score_list[-1]
                ))

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(questions))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_choices, predicted_choices)))
        print("Test Precision: {}".format(precision_score(golden_choices, predicted_choices, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_choices, predicted_choices, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_choices, predicted_choices, average="weighted")))
        print("Test classification Report:\n{}".format(
            classification_report(golden_choices, predicted_choices, digits=10)))

        total = len(exact_score_list)
        evaluation_results = collections.OrderedDict(
            [
                ("exact", 100.0 * sum(exact_score_list) / total),
                ("f1", 100.0 * sum(f1_score_list) / total),
                ("total", total),
            ]
        )
        print("evaluation results:\n", evaluation_results)

        return output_predictions


In [6]:
model_name='persiannlp/mt5-base-parsinlu-multiple-choice'
mcqa_model = MultipleChoiceQA(model_name=model_name, model_type="mt5")
print(mcqa_model.config)

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


MT5Config {
  "_name_or_path": "/home/patrick/hugging_face/t5/mt5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.7.0",
  "use_cache": true,
  "vocab_size": 250112
}



## Sample Inference

In [7]:
question_list = [
    "وسیع ترین کشور جهان کدام است؟",
    "طامع یعنی ؟",
    "زمینی به ۳۱ قطعه متساوی مفروض شده است و هر روز مساحت آماده شده برای احداث، دو برابر مساحت روز قبل است.اگر پس از (۵ روز) تمام زمین آماده شده باشد، در چه روزی یک قطعه زمین آماده شده"
]
candidate_list=[
    ["آمریکا", "کانادا", "روسیه", "چین"],
    ["آزمند", "خوش شانس", "محتاج", "مطمئن"],
    ["روز اول", "روز دوم", "روز سوم", "هیچکدام"]
]
mcqa_model.mt5_multiple_choice_qa_inference(question_list, candidate_list, device)

[('وسیع ترین کشور جهان کدام است؟',
  ['آمریکا', 'کانادا', 'روسیه', 'چین'],
  'چین'),
 ('طامع یعنی ؟', ['آزمند', 'خوش شانس', 'محتاج', 'مطمئن'], 'محتاج'),
 ('زمینی به ۳۱ قطعه متساوی مفروض شده است و هر روز مساحت آماده شده برای احداث، دو برابر مساحت روز قبل است.اگر پس از (۵ روز) تمام زمین آماده شده باشد، در چه روزی یک قطعه زمین آماده شده',
  ['روز اول', 'روز دوم', 'روز سوم', 'هیچکدام'],
  'روز اول')]

## Multiple-Choice Dataset


In [8]:
!git clone https://github.com/persiannlp/parsinlu
!ls parsinlu
!ls parsinlu/data/multiple-choice/

Cloning into 'parsinlu'...
remote: Enumerating objects: 1434, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 1434 (delta 110), reused 139 (delta 82), pack-reused 1252
Receiving objects: 100% (1434/1434), 27.81 MiB | 16.91 MiB/s, done.
Resolving deltas: 100% (913/913), done.
data  LICENSE  README.md  requirements.txt  scripts  src
test_ck.jsonl  test_lit.jsonl  train.jsonl
test.jsonl     test_ml.jsonl   valid.jsonl


### Samples with literature as their category

In [9]:
test_questions_lit, test_candidates_lit, test_choices_lit, test_answers_lit = mcqa_model.load_dataset_test_file(
    dataset_name="parsinlu-literature", dataset_file="./parsinlu/data/multiple-choice/test_lit.jsonl")
print(test_questions_lit[0])
print(test_candidates_lit[0])
print(test_choices_lit[0])
print(test_answers_lit[0])
print(len(test_questions_lit))
print(len(test_candidates_lit))
print(len(test_choices_lit))
print(len(test_answers_lit))

رابطه‌ی شیر با جنگل مثل رابطه‌ی
['سرباز است با پادگان', 'اتوبوس است با ایستگاه', 'هواپیما است با آسمان', 'کشتی است با بندر']
3
هواپیما است با آسمان
350
350
350
350


In [10]:
!nvidia-smi
!lscpu

Sat Aug 14 14:17:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    74W / 149W |   2789MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
evaluation_output = mcqa_model.mt5_evaluation(test_questions_lit, test_candidates_lit, test_choices_lit, test_answers_lit, device, max_length=512, batch_size=128)

#question:350, #candidates:350, #answer:350
#batch: 3
Start to evaluate test data ...
inference time for step 0: 16.82330255399995
inference time for step 1: 16.626356681999937
inference time for step 2: 12.463789421000001
total inference time: 45.91344865699989
total inference time / #samples: 0.13118128187714254
Test Accuracy: 0.38571428571428573
Test Precision: 0.38585234585234585
Test Recall: 0.38571428571428573
Test F1-Score(weighted average): 0.3855496289424861
Test classification Report:
              precision    recall  f1-score   support

           1  0.3333333333 0.3333333333 0.3333333333        75
           2  0.4215686275 0.4215686275 0.4215686275       102
           3  0.3838383838 0.4086021505 0.3958333333        93
           4  0.3918918919 0.3625000000 0.3766233766        80

    accuracy                      0.3857142857       350
   macro avg  0.3826580591 0.3815010278 0.3818396677       350
weighted avg  0.3858523459 0.3857142857 0.3855496289       350

evaluati

In [12]:
for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value))

رابطه‌ی شیر با جنگل مثل رابطه‌ی	['سرباز است با پادگان', 'اتوبوس است با ایستگاه', 'هواپیما است با آسمان', 'کشتی است با بندر']	3	هواپیما است با آسمان	3	هواپیما است با آسمان	1	1.0
رابطه ي بخار با یخ مثل رابطه ي:	['خمیر است با نان', 'گندم است با آرد', 'غوره است با کشمش', 'باران است با برف']	3	غوره است با کشمش	3	غوره است با کشمش	1	1.0
در عبارت زیر، به‌ترتیب « مضاف‌الیه مضاف‌الیه، صفت مضاف‌الیه و متمم اسم» کدام است؟
«مطالعه تفاسیر قرآن، روح اشعار حافظ شیراز را جلایی خاص بخشیده و از غزلیات این شاعر بی‌بدیل می‌توان به مهارت
	['قرآن، این، کشف', 'حافظ، عرفانی، غزلیات', 'شیراز، این شاعر، مهارت', 'رموز، بی\u200cبدیل، کشف رموز عرفانی']	1	قرآن، این، کشف	1	قرآن، این، کشف	1	1.0
مشهورترین شاعر رمانتیک قرن نوزدهم فرانسه چه کسی است؟	['ولتر', 'ویکتورهوگو', 'لافونتن', 'ژان ژاک  روسو']	2	ویکتورهوگو	1	ولتر	0	0
مفرد كدام كلمه صحيح است	['الوان : لون', 'حواس : احساس', 'اعضا: عضوها', 'الف وب']	3	اعضا: عضوها	1	الوان : لون	0	0
کدام عبارت، نادرست است؟	['شاعر منظومه\u200cهای حماسی مصنوع، با داستان\u200cهای پهلوانی م

In [13]:
output_file_name = "multiple_choice_qa_literature_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Samples with math_and_logic as their category

In [14]:
test_questions_ml, test_candidates_ml, test_choices_ml, test_answers_ml = mcqa_model.load_dataset_test_file(
    dataset_name="parsinlu-math_and_logic", dataset_file="./parsinlu/data/multiple-choice/test_ml.jsonl")
print(test_questions_ml[0])
print(test_candidates_ml[0])
print(test_choices_ml[0])
print(test_answers_ml[0])
print(len(test_questions_ml))
print(len(test_candidates_ml))
print(len(test_choices_ml))
print(len(test_answers_ml))

تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟
['2A', '2A+B', '3A+B', 'A-B']
2
2A+B
350
350
350
350


In [15]:
!nvidia-smi
!lscpu

Sat Aug 14 14:18:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    83W / 149W |   8935MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
evaluation_output = mcqa_model.mt5_evaluation(test_questions_ml, test_candidates_ml, test_choices_ml, test_answers_ml, device, max_length=512, batch_size=128)

#question:350, #candidates:350, #answer:350
#batch: 3
Start to evaluate test data ...
inference time for step 0: 16.210131040999954
inference time for step 1: 16.556340665999983
inference time for step 2: 12.44238550099999
total inference time: 45.20885720799993
total inference time / #samples: 0.12916816345142837
Test Accuracy: 0.4057142857142857
Test Precision: 0.415560059687854
Test Recall: 0.4057142857142857
Test F1-Score(weighted average): 0.40822719430343607
Test classification Report:
              precision    recall  f1-score   support

           1  0.5000000000 0.4152542373 0.4537037037       118
           2  0.4226804124 0.4141414141 0.4183673469        99
           3  0.3366336634 0.4197530864 0.3736263736        81
           4  0.3333333333 0.3461538462 0.3396226415        52

    accuracy                      0.4057142857       350
   macro avg  0.3981618523 0.3988256460 0.3963300164       350
weighted avg  0.4155600597 0.4057142857 0.4082271943       350

evaluation 

In [17]:
for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value))

تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟	['2A', '2A+B', '3A+B', 'A-B']	2	2A+B	2	2A+B	1	1.0
در ادامه این رشته چه عددی باید نوشت؟ ۹۱،۸۶،۷۶،۶۱،...	['۴۶', '۴۱', '۵۱', '۳۶']	2	۴۱	2	۲۱	0	0
50 تا 20 تا برابر است با ......	['10000', '100', '1000', '500']	3	1000	3	1000	1	1.0
در ادامه این رشته چه عددی باید نوشت؟             3, 5, 5, 9, 7, 13, 9, …	['17', '11', '14', '15']	1	17	3	14	0	0
مساحت مربع ۸ ،p برابر مساحت مربع Q است. نسبت قطر مربع p به ضلع مربع Q کدامست؟	['۴', '۲', '۳', '۱']	1	۴	4	۱	0	0
%50 عدد 24 برابر است با ....	['4', '6', '10', '12']	4	12	2	6	0	0
کدام عدد نزدیکتر۷ به است؟	['۴', '۶', '۹', '۱۱']	2	۶	3	۹	0	0
چند درصد ۵۰۰ برابر ۵۰ می‌شود؟	['۱', '۱۰', '۲۰', '۳۰']	2	۱۰	4	۳۰	0	0
قیمت یک کالا %۲۵ تخفیف داده شده است برای آنکه این کالا به قیمت قبل از تخفیف فروخته شود چند درصد باید به قیمت آن افزوده گردد؟	['۲۵', '۲۰', '۳۳.۳۳', 'هیچکدام']	3	۳۳.۳۳	3	۳۳.۳۳	1	1.0
حاصل عبارت ۵ - ۳ برابر است با ؟	['-2', '2', '1', '-1']	1	-2	1

In [18]:
output_file_name = "multiple_choice_qa_math_and_logic_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Samples with common_knowledge as their category

In [19]:
test_questions_ck, test_candidates_ck, test_choices_ck, test_answers_ck = mcqa_model.load_dataset_test_file(
    dataset_name="parsinlu-common_knowledge", dataset_file="./parsinlu/data/multiple-choice/test_ck.jsonl")
print(test_questions_ck[0])
print(test_candidates_ck[0])
print(test_choices_ck[0])
print(test_answers_ck[0])
print(len(test_questions_ck))
print(len(test_candidates_ck))
print(len(test_choices_ck))
print(len(test_answers_ck))

کدام کشور اولین تولید کننده خرما در جهان است؟
['ایران', 'عربستان', 'عراق', 'سوریه']
1
ایران
350
350
350
350


In [20]:
!nvidia-smi
!lscpu

Sat Aug 14 14:19:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    83W / 149W |   8935MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
evaluation_output = mcqa_model.mt5_evaluation(test_questions_ck, test_candidates_ck, test_choices_ck, test_answers_ck, device, max_length=512, batch_size=128)

#question:350, #candidates:350, #answer:350
#batch: 3
Start to evaluate test data ...
inference time for step 0: 16.52870419099986
inference time for step 1: 16.55262644300001
inference time for step 2: 12.407136788000116
total inference time: 45.488467421999985
total inference time / #samples: 0.12996704977714282
Test Accuracy: 0.24571428571428572
Test Precision: 0.24580062794348512
Test Recall: 0.24571428571428572
Test F1-Score(weighted average): 0.24551789908460653
Test classification Report:
              precision    recall  f1-score   support

           1  0.2500000000 0.2346938776 0.2421052632        98
           2  0.2142857143 0.2045454545 0.2093023256        88
           3  0.2916666667 0.2978723404 0.2947368421        94
           4  0.2179487179 0.2428571429 0.2297297297        70

    accuracy                      0.2457142857       350
   macro avg  0.2434752747 0.2449922038 0.2439685401       350
weighted avg  0.2458006279 0.2457142857 0.2455178991       350

evaluat

In [22]:
for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value))

کدام کشور اولین تولید کننده خرما در جهان است؟	['ایران', 'عربستان', 'عراق', 'سوریه']	1	ایران	1	ایران	1	1.0
مقام رهبری در قانون اساسی جمهوری اسلامی دارای چه کار ویژه ای است؟	['ریاست کشور', 'نظارت عالیه', 'تنظیم کننده قوای سه گانه', 'حاکمیت مطلق']	3	تنظیم کننده قوای سه گانه	3	تنظیم کننده قوای سه گانه	1	1.0
الماس سخت تر است یا گرانیت؟	['الماس', 'گرانیت', '', '']	1	الماس	2	گرانیت	0	0
طبق قانون اساسی شورای نگهبان طی چند روز از تاریخ وصول باید نظر خود را نسبت به مصوبات مجلس
	['یک ماه', 'ده روز', 'دو هفته', 'تا حصول اطمینان']	2	ده روز	3	دو هفته	0	0
در کشور ایران بیشترین نرخ بیکاری متعلق به کدام یک از گزینه های زیر میباشد؟	['دارندگان مدرک تحصیلی تکمیلی', 'افراد زیر دیپلم', 'افراد بالای 50 سال', 'فارغ التحصیلان دانشگاهها']	1	دارندگان مدرک تحصیلی تکمیلی	3	افراد بالای 50 سال	0	0
ریاست اولین دوره مجلس شورای اسلامی بر عهده چه کسی بود؟	['آیت الله بهشتی', 'آیت الله کروبی', 'آیت الله رفسنجانی', 'آیت الله ناطق نوری']	4	آیت الله ناطق نوری	2	آیت الله کروبی	0	0.5714285714285715
سوره بیست و هفتم قرآن کریم ک

In [23]:
output_file_name = "multiple_choice_qa_common_knowledge_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### All Samples

In [24]:
test_questions_all, test_candidates_all, test_choices_all, test_answers_all = mcqa_model.load_dataset_test_file(
    dataset_name="parsinlu", dataset_file="./parsinlu/data/multiple-choice/test.jsonl")
print(test_questions_all[0])
print(test_candidates_all[0])
print(test_choices_all[0])
print(test_answers_all[0])
print(len(test_questions_all))
print(len(test_candidates_all))
print(len(test_choices_all))
print(len(test_answers_all))

تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟
['2A', '2A+B', '3A+B', 'A-B']
2
2A+B
1050
1050
1050
1050


In [25]:
!nvidia-smi
!lscpu

Sat Aug 14 14:20:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    82W / 149W |   8935MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
evaluation_output = mcqa_model.mt5_evaluation(test_questions_all, test_candidates_all, test_choices_all, test_answers_all, device, max_length=512, batch_size=128)

#question:1050, #candidates:1050, #answer:1050
#batch: 9
Start to evaluate test data ...
inference time for step 0: 16.22692546600001
inference time for step 1: 16.513621458999978
inference time for step 2: 16.58862887600003
inference time for step 3: 16.551200136999796
inference time for step 4: 16.560216886000035
inference time for step 5: 16.55131984700006
inference time for step 6: 16.54500222699994
inference time for step 7: 16.539657833999854
inference time for step 8: 3.7818631860000096
total inference time: 135.8584359179997
total inference time / #samples: 0.12938898658857115
Test Accuracy: 0.3457142857142857
Test Precision: 0.34664509275773847
Test Recall: 0.3457142857142857
Test F1-Score(weighted average): 0.3457275345766356
Test classification Report:
              precision    recall  f1-score   support

           1  0.3660377358 0.3333333333 0.3489208633       291
           2  0.3604240283 0.3529411765 0.3566433566       289
           3  0.3378378378 0.3731343284 0.354

In [27]:
for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value))

تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟	['2A', '2A+B', '3A+B', 'A-B']	2	2A+B	2	2A+B	1	1.0
در ادامه این رشته چه عددی باید نوشت؟ ۹۱،۸۶،۷۶،۶۱،...	['۴۶', '۴۱', '۵۱', '۳۶']	2	۴۱	2	۲۱	0	0
50 تا 20 تا برابر است با ......	['10000', '100', '1000', '500']	3	1000	3	1000	1	1.0
در ادامه این رشته چه عددی باید نوشت؟             3, 5, 5, 9, 7, 13, 9, …	['17', '11', '14', '15']	1	17	3	14	0	0
مساحت مربع ۸ ،p برابر مساحت مربع Q است. نسبت قطر مربع p به ضلع مربع Q کدامست؟	['۴', '۲', '۳', '۱']	1	۴	4	۱	0	0
%50 عدد 24 برابر است با ....	['4', '6', '10', '12']	4	12	2	6	0	0
کدام عدد نزدیکتر۷ به است؟	['۴', '۶', '۹', '۱۱']	2	۶	3	۹	0	0
چند درصد ۵۰۰ برابر ۵۰ می‌شود؟	['۱', '۱۰', '۲۰', '۳۰']	2	۱۰	4	۳۰	0	0
قیمت یک کالا %۲۵ تخفیف داده شده است برای آنکه این کالا به قیمت قبل از تخفیف فروخته شود چند درصد باید به قیمت آن افزوده گردد؟	['۲۵', '۲۰', '۳۳.۳۳', 'هیچکدام']	3	۳۳.۳۳	3	۳۳.۳۳	1	1.0
حاصل عبارت ۵ - ۳ برابر است با ؟	['-2', '2', '1', '-1']	1	-2	1

In [28]:
output_file_name = "multiple_choice_qa_all_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer, exact_value, f1_value))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()